In [ ]:
# prepare env(shell command)

# conda create -n agent python=3.10
# conda activate agent
# conda install jupyter 
# pip install ipykernel
# python -m ipykernel install --user --name agent --display-name agent

# pip install -U "qwen-agent[gui,rag,code_interpreter,mcp]"

In [1]:
import os

def load_api_key(file_path: str):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
    except FileNotFoundError:
        raise Exception("API key file not found")

In [2]:
# define the function to query weather data
import requests

def get_weather(city: str) -> str:
    """
    获取实时天气
    
    Args:
        city (str): 城市名
        
    Returns:
        str: 实时天气信息
    """
    
    # get the location ID by using the Geo API
    geo_url = "https://geoapi.qweather.com/v2/city/lookup"
    geo_params = {
        "location": city,
        "key": load_api_key("qweather_key.txt")
    }
    
    try:
        geo_response = requests.get(geo_url, params=geo_params)
        geo_data = geo_response.json()
        
        if geo_data.get("code") != "200" or not geo_data.get("location"):
            return f"Error: Could not find location ID for {city}"
        
        # extract the location ID from the first result
        location_id = geo_data["location"][0]["id"]
        city_name = geo_data["location"][0]["name"]
        
        # query the weather with the location ID
        weather_url = "https://api.qweather.com/v7/weather/now"
        weather_params = {
            "location": location_id,  # Using location_id instead of city name
            "key": load_api_key("qweather_key.txt"),
            "lang": "zh"
        }
        
        weather_response = requests.get(weather_url, params=weather_params)
        weather_data = weather_response.json()
        
        if weather_data.get("code") == "200":
            weather_info = weather_data.get("now", {})
            result = f"{city_name}实时天气：\n"
            result += f"温度：{weather_info.get('temp')}°C\n"
            result += f"天气：{weather_info.get('text')}\n"
            result += f"风向：{weather_info.get('windDir')}\n"
            result += f"风速：{weather_info.get('windSpeed')} km/h\n"
            result += f"相对湿度：{weather_info.get('humidity')}%"
            return result
        else:
            return f"Error: {weather_data.get('code')} - {weather_data.get('message', 'Unknown error')}"
    
    except Exception as e:
        return f"Failed to get weather data: {str(e)}"


In [3]:
# test the function
print(get_weather('北京'))

北京实时天气：
温度：13°C
天气：晴
风向：北风
风速：33 km/h
相对湿度：19%


In [4]:
# register tool for llm
import urllib.parse
import json5
import json
from qwen_agent.tools.base import BaseTool, register_tool

@register_tool('weather_query')
class WeatherQuery(BaseTool):
    description = '获得城市的实时天气'
    parameters = [{
        'name': 'city',
        'type': 'string',
        'description': '城市',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        city = json5.loads(params)['city']
        return get_weather(city)

In [5]:
from qwen_agent.agents import Assistant

llm_cfg = {
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': load_api_key("key.txt")
}

bot = Assistant(
    llm = llm_cfg,
    system_message = '你是一个能获得实时天气的AI助手',
    function_list = ['weather_query'] 
)

In [6]:
from qwen_agent.gui import WebUI

WebUI(bot).run(server_port=7860)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
